In [1]:
import langchain_community
import os

# Load Data

In [2]:
from langchain_community.document_loaders import DirectoryLoader

from dotenv import load_dotenv

load_dotenv(override=True)


# loader = DirectoryLoader("data", glob="*.md", show_progress=True)
# docs = loader.load()

# if docs:
# 	print(docs[0].page_content[:100])
# else:
# 	print("No documents found.")



True

def combine(pdf_folder_path):
    documents = []
    for file in os.listdir(pdf_folder_path):
        if file.endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder_path, file)
            loader = PyPDFLoader(pdf_path)
            documents.extend(loader.load())
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(documents)
    return splits

splits = load_and_process_pdfs(pdf_folder_path)

# RAG Framework

chunking & emdedding 

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import Document
from langchain.schema import Document
import os
import shutil

data_dir = "input/"
CHROMA_PATH = "chroma"

def load_and_process_markdown(md_folder_path):
    documents = []
    for file in os.listdir(md_folder_path):
        if file.endswith('.md'):
            md_path = os.path.join(md_folder_path, file)
            with open(md_path, "r", encoding="utf-8") as f:
                content = f.read()
            documents.append(Document(page_content=content, metadata={"source": md_path}))
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(documents)
    print("length of document: ", len(documents))
    combined_docs_path = "combined_documents.md"
    with open(combined_docs_path, "w", encoding="utf-8") as f:
        for doc in documents:
            f.write(doc.page_content + "\n\n")
    return splits

chunks = load_and_process_markdown(data_dir)

# embedding
if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)

vectorstore = Chroma.from_documents(
        chunks, OpenAIEmbeddings(), persist_directory=CHROMA_PATH
)

print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")


length of document:  3
Saved 403 chunks to chroma.


Initializing the RAG chain

In [2]:
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

llm = ChatOpenAI(model="gpt-4o-mini")

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# test logs
print(retriever | format_docs)
print(retriever)
print(format_docs)



e:\Repositories\graphrag-businessqa-evaluation\env\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


first=VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000002B04492CD10>, search_kwargs={}) middle=[] last=RunnableLambda(format_docs)
tags=['Chroma', 'OpenAIEmbeddings'] vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000002B04492CD10> search_kwargs={}
<function format_docs at 0x000002B041E78680>


# Preparing Evaluation

map generated questions and ground_truths into evaluation dataset

In [ ]:
import json
from datasets import Dataset

with open('synthetic_data_big_context.json', 'r') as f:
    synthetic_data = json.load(f)

if isinstance(synthetic_data, dict) and 'responses' in synthetic_data:
    synthetic_data = synthetic_data['responses']

questions = [item.get('question', "") for item in synthetic_data]
ground_truths = [item.get('ground_truth', "") for item in synthetic_data]
reference = [item.get('context', "") for item in synthetic_data]

answers = []
retrieved_contexts = []

for query in questions:

    answer = rag_chain.invoke(query)
    answers.append(answer)
    print(answer)
    retrieved_context = [doc.page_content for doc in retriever.get_relevant_documents(query)]
    retrieved_contexts.append(retrieved_context)

data = {
    "question": questions,
    "answer": answers,
    "retrieved_contexts": retrieved_contexts, 
    "ground_truth": ground_truths,
    "reference": reference 
}

dataset = Dataset.from_dict(data)

print(dataset)


SAP can enhance customer experience and drive operational efficiency by integrating Business AI across its product portfolio, making AI relevant, reliable, and responsible. By leveraging its extensive data assets from over 25,000 customer tenants, SAP can create AI solutions tailored to complex business processes. This approach not only provides real-time, truthful results but also transforms SAP's offerings into an AI-powered ecosystem that delivers significant business value.
SAP can enhance its transition to a cloud-first model by integrating sustainability data into its core ERP system, allowing customers to optimize both sustainability and financial performance. By leveraging cloud-driven innovations and AI-powered sustainability use cases, SAP can provide tools that enable agility and responsiveness to evolving sustainability challenges. Additionally, focusing on process excellence and automation will ensure efficient operations and support long-term sustainability strategies for

In [26]:
print(dataset[0])

{'question': 'How can SAP leverage its extensive data assets and AI capabilities to enhance customer experience while simultaneously driving operational efficiency across its product portfolio?', 'answer': 'SAP can enhance customer experience and drive operational efficiency by infusing Business AI across its product portfolio, ensuring that AI is relevant, reliable, and responsible. By leveraging unique data assets from over 25,000 customer tenants, SAP can create tailored AI solutions that integrate seamlessly into critical business processes. This approach not only delivers immediate value but also supports customers in achieving their broader goals, such as sustainability and industry transformation.', 'retrieved_context': "SAP is uniquely positioned to help our customers benefit from artificial intelligence by delivering Business AI, which we define as:\n- **Relevant:** AI delivered, from day one, across all SAP applications that customers use to run their businesses.\n- **Reliabl

# Evaluation RAG

In [33]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    )

print(dataset)

result = evaluate(
    dataset = dataset, 
    metrics=[
        faithfulness,
        answer_relevancy,
        context_recall,
        context_precision
    ],
)

df = result.to_pandas()

Dataset({
    features: ['question', 'answer', 'retrieved_contexts', 'ground_truth', 'reference'],
    num_rows: 14
})


Evaluating: 100%|██████████| 56/56 [00:28<00:00,  1.93it/s]


# Results

In [38]:
import pandas as pd
pd.set_option('display.max_colwidth', 50) # default is 50 / None for unlimited

df
 


,user_input,retrieved_contexts,response,reference,faithfulness,answer_relevancy,context_recall,context_precision
0,How can SAP leverage its extensive data assets...,[SAP is uniquely positioned to help our custom...,SAP can enhance customer experience and drive ...,"SAP has a vast repository of customer data, wh...",0.888889,0.951106,0.666667,1.000000
1,What strategies can SAP implement to ensure th...,[The speed of cloud delivery combined with the...,SAP can enhance its transition to a cloud-firs...,SAP's commitment to sustainability is evident ...,0.875000,0.949602,0.666667,1.000000
2,How can SAP utilize generative AI to enhance i...,[## Infuse Business AI across the SAP product ...,SAP can leverage generative AI to enhance prod...,The integration of generative AI into product ...,1.000000,0.960907,0.666667,1.000000
3,In what ways can SAP's Business Technology Pla...,"[As an industry, we are benefiting from a secu...",SAP's Business Technology Platform (BTP) foste...,BTP serves as the backbone for SAP's cloud sol...,0.384615,0.954293,0.666667,1.000000
4,How can SAP align its AI initiatives with its ...,[The speed of cloud delivery combined with the...,SAP can align its AI initiatives with corporat...,SAP's commitment to sustainability is a core a...,1.000000,0.984032,1.000000,0.916667
5,What role does SAP's partner ecosystem play in...,[- **Unleash SAP’s ecosystem.** SAP’s more tha...,"SAP's partner ecosystem, comprising over 25,50...",The partner ecosystem is integral to SAP's str...,0.636364,0.944183,1.000000,1.000000
6,How can SAP ensure that its AI-driven solution...,[- **Focus on the cloud.** Modern AI happens i...,SAP ensures its AI-driven solutions are effect...,SAP's commitment to responsible AI aligns with...,0.833333,0.976844,0.666667,1.000000
7,What innovative approaches can SAP adopt to en...,[### Enhance\nEnhance Product Quality and Cust...,SAP can enhance its customer support services ...,The integration of AI into customer support al...,1.000000,0.932442,1.000000,1.000000
8,How can SAP's focus on sustainability be integ...,[## SAP Solutions\n- SAP sustainability and ES...,SAP can integrate sustainability into its prod...,SAP's commitment to sustainability is a key as...,0.888889,0.975433,0.666667,1.000000
9,What strategies can SAP employ to enhance its ...,[- **Business AI** is the defining technology ...,SAP can enhance its market position by infusin...,The competitive landscape in the AI and cloud ...,0.333333,0.940235,0.666667,1.000000


# GraphRAG

## Indexing

In [ ]:
# graphrag setup
!mkdir -p ./graphrag
!python -m graphrag init --root ./graphrag

The syntax of the command is incorrect.




⠋ GraphRAG Indexer 
Initializing project at E:\Repositories\graphrag-businessqa-evaluation\graphrag
⠋ GraphRAG Indexer 


┌───────────────────── Traceback (most recent call last) ─────────────────────┐
│ e:\Repositories\graphrag-businessqa-evaluation\env\Lib\site-packages\graphr │
│ ag\cli\main.py:57 in _initialize_cli                                        │
│                                                                             │
│    54 │   ],                                                                │
│    55 ):                                                                    │
│    56 │   """Generate a default configuration file."""                      │
│ >  57 │   initialize_project_at(path=root)                                  │
│    58                                                                       │
│    59                                                                       │
│    60 @app.command("index")                                                 │
│                                                                             │
│ ┌──────────────────────────────── loca

In [ ]:
# indexing, only run once unless you want to update the index
# !graphrag index --root ./graphrag



⠋ GraphRAG Indexer 
Logging enabled at 
E:\Repositories\graphrag-businessqa-evaluation\graphrag\logs\indexing-engine.lo
g
⠋ GraphRAG Indexer 
⠋ GraphRAG Indexer 
⠹ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (1 filtered) - 100%  0…
⠹ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (1 filtered) - 100%  0…
└── create_base_text_units
⠋ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (1 filtered) - 100%  0…
└── create_base_text_units
🚀 create_base_text_units
⠙ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (1 filtered) - 100%  0…
└── create_base_text_units
Empty DataFrame
Columns: []
Index: []
⠙ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (1 filtered) - 100%  0…
└── create_base_text_units
⠹ GraphRAG Indexer 
├── Loading Input (InputFileType.text) - 1 files loaded (1 filtered) - 100%  0…
└── create_base_text_units
⠹ GraphRAG Indexer 
├── Loading Input 

[2024-11-14T01:38:05Z WARN  lance::dataset] No existing dataset at E:\Repositories\graphrag-businessqa-evaluation\graphrag\output\lancedb\default-entity-description.lance, it will be created
[2024-11-14T01:38:07Z WARN  lance::dataset] No existing dataset at E:\Repositories\graphrag-businessqa-evaluation\graphrag\output\lancedb\default-community-full_content.lance, it will be created
[2024-11-14T01:38:10Z WARN  lance::dataset] No existing dataset at E:\Repositories\graphrag-businessqa-evaluation\graphrag\output\lancedb\default-text_unit-text.lance, it will be created


## Query Engine

### Global Search

follows the implementations guide by the docs https://microsoft.github.io/graphrag/examples_notebooks/global_search/

In [4]:
import os

import pandas as pd
import tiktoken

from graphrag.query.indexer_adapters import (
    read_indexer_entities,
    read_indexer_reports,
)
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch

In [ ]:
api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = "gpt-4o-mini"

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.OpenAI,  
    max_retries=20,
)

token_encoder = tiktoken.encoding_for_model(llm_model)

In [ ]:
INPUT_DIR = "./input"
OUTPUT_DIR = "./graphrag/output/"

COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"


COMMUNITY_LEVEL = 2

In [ ]:
#community_df = pd.read_parquet(f"{OUTPUT_DIR}/{COMMUNITY_TABLE}.parquet")
entity_df = pd.read_parquet(f"{OUTPUT_DIR}/{ENTITY_TABLE}.parquet")
report_df = pd.read_parquet(f"{OUTPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{OUTPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

#communities = read_indexer_communities(community_df, entity_df, report_df)
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)

print(f"Total report count: {len(report_df)}")
print(
    f"Report count after filtering by community level {COMMUNITY_LEVEL}: {len(reports)}"
)
# Keeping pandas from truncating long strings
# pd.set_option('display.max_colwidth', 50) default is 50 / None for unlimited
report_df.head()

Total report count: 34
Report count after filtering by community level 2: 27


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id,size,period
0,17,# Generative AI Community Impact Report\n\nThe...,1,8.5,Generative AI Community Impact Report,The impact severity rating is high due to the ...,The community surrounding Generative AI encomp...,[{'explanation': 'Generative AI is poised to r...,"{\n ""title"": ""Generative AI Community Impac...",91666503-1d20-4d41-95ef-1fad4da149c1,27,2024-11-14
1,18,# Morgan Stanley and OpenAI Integration\n\nThe...,1,7.5,Morgan Stanley and OpenAI Integration,The impact severity rating is high due to the ...,"The community centers around Morgan Stanley, a...",[{'explanation': 'Morgan Stanley is at the for...,"{\n ""title"": ""Morgan Stanley and OpenAI Int...",25317729-b3b6-48c9-b069-bc88683ac0c3,2,2024-11-14
2,19,# McKinsey Global Institute and Generative AI ...,1,7.5,McKinsey Global Institute and Generative AI Im...,The impact severity rating is high due to the ...,The community centers around the McKinsey Glob...,[{'explanation': 'The McKinsey Global Institut...,"{\n ""title"": ""McKinsey Global Institute and...",9f8d42ee-8cc3-4c95-a210-5cd497d57638,6,2024-11-14
3,20,# McKinsey & Company and Generative AI Impact\...,1,8.5,McKinsey & Company and Generative AI Impact,The impact severity rating is high due to the ...,The community centers around McKinsey & Compan...,[{'explanation': 'McKinsey & Company is at the...,"{\n ""title"": ""McKinsey & Company and Genera...",349deccf-6948-4a8a-b577-c9956ea593c8,9,2024-11-14
4,21,# Generative AI and Its Impact on Workforce an...,1,8.5,Generative AI and Its Impact on Workforce and ...,The impact severity rating is high due to the ...,"The community centers around generative AI, wh...",[{'explanation': 'Generative AI is a significa...,"{\n ""title"": ""Generative AI and Its Impact ...",27cddb7c-652f-4efb-a22f-8abeb4b06acd,5,2024-11-14


In [ ]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    #communities=communities,
    entities=entities, 
    token_encoder=token_encoder,
)

global search

In [ ]:
context_builder_params = {
    "use_community_summary": False,  
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 2000,
    "temperature": 0.0,
}

In [ ]:
search_engine = GlobalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000, 
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=False, 
    json_mode=True, 
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",
)

In [20]:
result = await search_engine.asearch(
    "What is the estimated economic impact of generative AI on the global economy?"
)

print(result.response)

## Economic Impact of Generative AI on the Global Economy

Generative AI is projected to have a profound impact on the global economy, with estimates suggesting it may add trillions of dollars in value. This technology is expected to enhance productivity across various sectors, driving economic growth and reshaping the economic landscape. The transformative capabilities of generative AI are likely to lead to increased efficiency and innovation, which are critical for the overall economic dynamics [Data: Reports (26)].

### Key Drivers of Economic Growth

The anticipated economic benefits of generative AI stem from its ability to create new product categories and streamline processes. By automating tasks, generative AI is expected to open up new markets and significantly contribute to overall economic development. The relationship between generative AI and economic performance indicates that technological advancements will play a crucial role in shaping future economic trends [Data: Rep

In [ ]:
pd.set_option('display.max_colwidth', 50) # default is 50 / None for unlimited

result.context_data["reports"]

NameError: name 'pd' is not defined

In [ ]:
print(
    f"LLM calls: {result.llm_calls}. Prompt tokens: {result.prompt_tokens}. Completion time: {result.completion_time}."
)

print(result)

NameError: name 'result' is not defined